# 🧭 꿀스테이 RAG - 라우팅 및 통합

이 노트북에서는 질문 분석, 에이전트 라우팅, 멀티 에이전트 답변 통합을 구현합니다.

## 목표
1. 질문 분석 및 도메인 분류 시스템
2. 적절한 에이전트 선택 로직
3. 멀티 에이전트 답변 조합
4. 마스터 오케스트레이션 구현
5. LangGraph 워크플로우 구성

In [ ]:
import os
import sys
from pathlib import Path
from typing import List, Dict, Any, Optional, Tuple, Union
import logging
from dotenv import load_dotenv
from dataclasses import dataclass, field
from enum import Enum
import json
from datetime import datetime

# LangChain 관련
from langchain_openai import ChatOpenAI
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.documents import Document
from langchain_core.runnables import RunnablePassthrough

# LangGraph 관련
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver
from typing_extensions import TypedDict

# 웹 검색
from langchain_community.tools import TavilySearchResults

# 환경변수 로드 (절대 경로 지정)
project_root = Path("/Users/yundoun/Desktop/Project/legal_rag/coolstay_rag")
env_file = project_root / ".env"
load_result = load_dotenv(env_file)
print(f"📝 .env 파일 로드: {load_result} (경로: {env_file})")

# API 키 확인
openai_key = os.getenv("OPENAI_API_KEY", "NOT_FOUND")
tavily_key = os.getenv("TAVILY_API_KEY", "NOT_FOUND")
print(f"🔑 OpenAI API Key: {'설정됨' if openai_key != 'NOT_FOUND' and openai_key.startswith('sk-') else 'NOT_FOUND'}")
print(f"🔑 Tavily API Key: {'설정됨' if tavily_key != 'NOT_FOUND' and tavily_key.startswith('tvly-') else 'NOT_FOUND'}")

# 로깅 설정
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print("✅ 라이브러리 import 완료")

## 1. 기본 설정 및 에이전트 로딩

In [ ]:
# 프로젝트 경로 설정
PROJECT_ROOT = Path("/Users/yundoun/Desktop/Project/legal_rag/coolstay_rag")
DATA_DIR = PROJECT_ROOT / "data"
CHROMA_DB_DIR = PROJECT_ROOT / "chroma_db"

# 도메인 설정
DOMAIN_CONFIG = {
    "hr_policy": {
        "file": "HR_Policy_Guide.md",
        "description": "인사정책, 근무시간, 휴가, 급여, 복리후생",
        "collection_name": "hr_policy_db",
        "agent_name": "HR 정책 전문가",
        "keywords": ["인사", "HR", "근무", "휴가", "연차", "급여", "보험", "복리후생", "채용", "퇴사", "승진"]
    },
    "tech_policy": {
        "file": "Tech_Policy_Guide.md",
        "description": "기술정책, 개발환경, 코딩표준, 보안정책",
        "collection_name": "tech_policy_db",
        "agent_name": "기술 정책 전문가",
        "keywords": ["기술", "개발", "코딩", "코드", "프로그래밍", "보안", "개발환경", "IDE", "도구", "라이브러리"]
    },
    "architecture": {
        "file": "Architecture_Guide.md",
        "description": "CMS 아키텍처, 시스템설계, 레이어구조",
        "collection_name": "architecture_db",
        "agent_name": "아키텍처 전문가",
        "keywords": ["아키텍처", "시스템", "설계", "구조", "레이어", "모듈", "서비스", "API", "데이터베이스", "인프라"]
    },
    "component": {
        "file": "Component_Guide.md",
        "description": "컴포넌트 가이드라인, UI/UX 표준",
        "collection_name": "component_db",
        "agent_name": "컴포넌트 개발 전문가",
        "keywords": ["컴포넌트", "UI", "UX", "인터페이스", "디자인", "프론트엔드", "사용자", "화면", "페이지"]
    },
    "deployment": {
        "file": "Deployment_Guide.md",
        "description": "배포프로세스, CI/CD, 환경관리",
        "collection_name": "deployment_db",
        "agent_name": "배포 전문가",
        "keywords": ["배포", "CI/CD", "환경", "서버", "클라우드", "도커", "쿠버네티스", "파이프라인", "자동화"]
    },
    "development": {
        "file": "Development_Process_Guide.md",
        "description": "개발프로세스, 워크플로우, 협업규칙",
        "collection_name": "development_db",
        "agent_name": "개발 프로세스 전문가",
        "keywords": ["개발프로세스", "워크플로우", "협업", "프로세스", "방법론", "스크럼", "애자일", "리뷰", "테스트"]
    },
    "business_policy": {
        "file": "Business_Policy_Guide.md",
        "description": "비즈니스정책, 운영규칙, 의사결정",
        "collection_name": "business_policy_db",
        "agent_name": "비즈니스 정책 전문가",
        "keywords": ["비즈니스", "정책", "운영", "의사결정", "전략", "계획", "목표", "성과", "관리", "조직"]
    },
    "web_search": {
        "description": "실시간 웹 검색을 통한 최신 정보 제공",
        "agent_name": "웹 검색 전문가",
        "keywords": ["최신", "트렌드", "뉴스", "동향", "현재", "실시간", "업데이트", "신기술", "외부정보"]
    }
}

print(f"✅ 설정 완료: {len(DOMAIN_CONFIG)}개 도메인 (웹검색 포함)")

In [ ]:
# 이전 노트북에서 구현한 클래스들 임포트 (간소화된 버전)
# 실제로는 03_agents_development.ipynb의 코드를 재사용하거나 모듈로 분리

# 간소화된 RAGResponse 클래스
@dataclass
class RAGResponse:
    answer: str
    source_documents: List[Document]
    domain: str
    quality_assessment: Optional[Any] = None
    corrective_iterations: int = 0
    confidence_score: float = 0.7  # 기본 신뢰도

# 간소화된 기본 에이전트
class MockRAGAgent:
    """테스트용 Mock RAG Agent"""
    
    def __init__(self, domain: str, llm: ChatOpenAI):
        self.domain = domain
        self.llm = llm
        self.config = DOMAIN_CONFIG.get(domain, {})
        self.agent_name = self.config.get("agent_name", f"{domain} 전문가")
        self.description = self.config.get("description", "도메인 전문가")
    
    def query(self, question: str, enable_corrective: bool = True) -> RAGResponse:
        """Mock 질문 처리 (실제 구현에서는 03번 노트북의 BaseRAGAgent 사용)"""
        try:
            # 간단한 답변 생성 (실제로는 벡터 검색 + LLM 체인 사용)
            prompt = f"당신은 꿀스테이의 {self.agent_name}입니다. 다음 질문에 {self.description} 관점에서 답변해주세요: {question}"
            
            answer = self.llm.invoke(prompt).content
            
            # Mock 소스 문서
            source_docs = [
                Document(
                    page_content=f"{self.domain} 도메인 관련 정보",
                    metadata={"domain": self.domain, "source": "mock_data"}
                )
            ]
            
            return RAGResponse(
                answer=answer,
                source_documents=source_docs,
                domain=self.domain,
                confidence_score=0.8
            )
            
        except Exception as e:
            logger.error(f"Mock 에이전트 질문 처리 실패 ({self.domain}): {e}")
            return RAGResponse(
                answer=f"죄송합니다. {self.domain} 관련 질문 처리 중 오류가 발생했습니다.",
                source_documents=[],
                domain=self.domain,
                confidence_score=0.1
            )

print("✅ Mock 클래스 정의 완료")

In [ ]:
# LLM 초기화
def initialize_llm():
    try:
        llm = ChatOpenAI(
            model="gpt-4o-mini",
            temperature=0.1,
            api_key=os.getenv("OPENAI_API_KEY")
        )
        
        # 테스트 호출
        test_response = llm.invoke("Hello")
        print(f"✅ LLM 초기화 성공: {llm.model_name}")
        return llm
        
    except Exception as e:
        print(f"❌ LLM 초기화 실패: {e}")
        return None

# Mock 에이전트들 생성 (실제 구현에서는 03번 노트북의 실제 에이전트 사용)
def create_mock_agents(llm: ChatOpenAI) -> Dict[str, MockRAGAgent]:
    """Mock 에이전트 생성"""
    agents = {}
    
    for domain in DOMAIN_CONFIG.keys():
        if domain != "web_search":  # 웹검색은 별도 처리
            agents[domain] = MockRAGAgent(domain, llm)
    
    print(f"✅ Mock 에이전트 생성 완료: {len(agents)}개")
    return agents

# 초기화
llm = initialize_llm()
if llm:
    mock_agents = create_mock_agents(llm)
else:
    mock_agents = {}
    print("❌ LLM이 초기화되지 않아 Mock 에이전트를 생성할 수 없습니다.")

## 2. 질문 분석 및 도메인 분류 시스템

In [ ]:
@dataclass
class QuestionAnalysis:
    """질문 분석 결과"""
    original_question: str
    question_type: str  # "specific", "general", "comparison", "multi_domain"
    relevant_domains: List[str]  # 관련 도메인 목록
    confidence_scores: Dict[str, float]  # 도메인별 신뢰도
    keywords_found: List[str]  # 발견된 키워드
    needs_web_search: bool  # 웹 검색 필요 여부
    priority_domains: List[str]  # 우선순위 도메인
    reasoning: str  # 분석 근거

class QuestionAnalyzer:
    """질문 분석 및 도메인 분류기"""
    
    def __init__(self, llm: ChatOpenAI, domain_config: Dict[str, Dict]):
        self.llm = llm
        self.domain_config = domain_config
        
        # 도메인 분석 프롬프트
        self.analysis_prompt = ChatPromptTemplate.from_template("""
        당신은 꿀스테이 회사의 질문을 분석하고 적절한 전문가 도메인을 선택하는 전문가입니다.
        
        **사용 가능한 도메인:**
        {domain_descriptions}
        
        **질문:** {question}
        
        **분석 요청:**
        1. 질문 유형 분류 (specific/general/comparison/multi_domain)
        2. 관련 도메인 식별 (1-3개 추천)
        3. 도메인별 관련도 점수 (0.0-1.0)
        4. 웹 검색 필요성 판단
        5. 우선순위 도메인 선정
        
        **분석 기준:**
        - 키워드 매칭을 통한 도메인 식별
        - 질문의 맥락과 의도 파악
        - 최신 정보가 필요한 경우 웹 검색 추천
        - 복합적인 질문의 경우 여러 도메인 선택
        
        다음 JSON 형식으로 응답하세요:
        {{
            "question_type": "specific|general|comparison|multi_domain",
            "relevant_domains": ["domain1", "domain2", ...],
            "confidence_scores": {{"domain1": 0.9, "domain2": 0.7, ...}},
            "keywords_found": ["keyword1", "keyword2", ...],
            "needs_web_search": true|false,
            "priority_domains": ["domain1", "domain2"],
            "reasoning": "분석 근거 설명"
        }}
        """)
        
        # 분석 체인 구성
        self.analysis_chain = (
            self.analysis_prompt
            | self.llm
            | JsonOutputParser()
        )
    
    def _build_domain_descriptions(self) -> str:
        """도메인 설명 문자열 생성"""
        descriptions = []
        
        for domain, config in self.domain_config.items():
            agent_name = config.get("agent_name", domain)
            description = config.get("description", "")
            keywords = config.get("keywords", [])
            
            desc_text = f"- {domain}: {agent_name} ({description})"
            if keywords:
                desc_text += f" [키워드: {', '.join(keywords[:5])}]"
            
            descriptions.append(desc_text)
        
        return "\n".join(descriptions)
    
    def analyze_question(self, question: str) -> QuestionAnalysis:
        """질문 분석 수행"""
        try:
            domain_descriptions = self._build_domain_descriptions()
            
            result = self.analysis_chain.invoke({
                "question": question,
                "domain_descriptions": domain_descriptions
            })
            
            return QuestionAnalysis(
                original_question=question,
                question_type=result.get("question_type", "general"),
                relevant_domains=result.get("relevant_domains", []),
                confidence_scores=result.get("confidence_scores", {}),
                keywords_found=result.get("keywords_found", []),
                needs_web_search=result.get("needs_web_search", False),
                priority_domains=result.get("priority_domains", []),
                reasoning=result.get("reasoning", "분석 완료")
            )
            
        except Exception as e:
            logger.error(f"질문 분석 실패: {e}")
            # 기본값 반환
            return QuestionAnalysis(
                original_question=question,
                question_type="general",
                relevant_domains=["hr_policy"],  # 기본 도메인
                confidence_scores={"hr_policy": 0.5},
                keywords_found=[],
                needs_web_search=False,
                priority_domains=["hr_policy"],
                reasoning="분석 과정에서 오류 발생, 기본값 사용"
            )

# 질문 분석기 초기화
if llm:
    question_analyzer = QuestionAnalyzer(llm, DOMAIN_CONFIG)
    print("✅ 질문 분석기 초기화 완료")
else:
    question_analyzer = None
    print("❌ LLM이 초기화되지 않아 질문 분석기를 생성할 수 없습니다.")

## 3. 질문 분석 테스트

In [ ]:
# 테스트 질문들
test_questions = [
    "연차 휴가는 어떻게 신청하나요?",  # hr_policy
    "코딩 스타일 가이드는 무엇인가요?",  # tech_policy
    "시스템 아키텍처와 배포 프로세스를 설명해주세요",  # multi_domain: architecture + deployment
    "2024년 AI 기술 트렌드는 무엇인가요?",  # web_search
    "회사의 개발 프로세스와 UI 컴포넌트 가이드라인을 알려주세요"  # multi_domain: development + component
]

def test_question_analysis(analyzer: QuestionAnalyzer, questions: List[str]):
    """질문 분석 테스트"""
    
    print("🔍 질문 분석 테스트 시작\n")
    
    for i, question in enumerate(questions, 1):
        print(f"📋 테스트 {i}: {question}")
        print("="*60)
        
        try:
            # 질문 분석 수행
            analysis = analyzer.analyze_question(question)
            
            # 결과 출력
            print(f"🏷️  질문 유형: {analysis.question_type}")
            print(f"🎯 관련 도메인: {', '.join(analysis.relevant_domains)}")
            print(f"⭐ 우선순위: {', '.join(analysis.priority_domains)}")
            print(f"🌐 웹 검색 필요: {'예' if analysis.needs_web_search else '아니오'}")
            
            # 신뢰도 점수
            if analysis.confidence_scores:
                print(f"📊 신뢰도 점수:")
                for domain, score in analysis.confidence_scores.items():
                    print(f"   - {domain}: {score:.2f}")
            
            # 발견된 키워드
            if analysis.keywords_found:
                print(f"🔤 키워드: {', '.join(analysis.keywords_found)}")
            
            print(f"💭 분석 근거: {analysis.reasoning}")
            print(f"✅ 분석 완료")
            
        except Exception as e:
            print(f"❌ 분석 실패: {e}")
        
        print("\n" + "="*70 + "\n")

# 질문 분석 테스트 실행
if question_analyzer:
    test_question_analysis(question_analyzer, test_questions[:3])  # 처음 3개만 테스트
else:
    print("❌ 질문 분석기가 초기화되지 않아 테스트를 건너뜁니다.")

## 4. 멀티 에이전트 답변 통합 시스템

In [ ]:
@dataclass
class IntegratedResponse:
    """통합된 최종 응답"""
    final_answer: str
    contributing_domains: List[str]
    individual_responses: Dict[str, RAGResponse]
    integration_method: str  # "single", "combined", "ranked"
    overall_confidence: float
    source_summary: List[str]
    processing_time: float

class ResponseIntegrator:
    """멀티 에이전트 응답 통합기"""
    
    def __init__(self, llm: ChatOpenAI):
        self.llm = llm
        
        # 단일 응답 선택 프롬프트
        self.single_selection_prompt = ChatPromptTemplate.from_template("""
        여러 전문가의 답변 중에서 질문에 가장 적합한 답변을 선택해주세요.
        
        **질문:** {question}
        
        **전문가 답변들:**
        {responses}
        
        **선택 기준:**
        1. 질문과의 직접적 관련성
        2. 답변의 구체성과 완성도
        3. 전문가의 신뢰도
        
        가장 적합한 답변을 선택하고, 필요시 약간의 수정을 가해 최종 답변을 제공하세요.
        """)
        
        # 복합 통합 프롬프트
        self.integration_prompt = ChatPromptTemplate.from_template("""
        여러 전문가의 답변을 통합하여 포괄적인 최종 답변을 만들어주세요.
        
        **질문:** {question}
        
        **전문가 답변들:**
        {responses}
        
        **통합 지침:**
        1. 각 전문가의 고유한 관점을 존중하여 통합
        2. 중복되는 내용은 정리하고 보완적인 내용은 조합
        3. 전문 분야별로 구분하여 체계적으로 구성
        4. 일관성 있고 자연스러운 흐름으로 작성
        5. 각 정보의 출처(전문가)를 명시
        
        통합된 최종 답변을 제공하세요.
        """)
        
        # 체인 구성
        self.single_chain = self.single_selection_prompt | self.llm | StrOutputParser()
        self.integration_chain = self.integration_prompt | self.llm | StrOutputParser()
    
    def _format_responses(self, responses: Dict[str, RAGResponse]) -> str:
        """응답들을 포맷팅"""
        formatted = []
        
        for domain, response in responses.items():
            domain_config = DOMAIN_CONFIG.get(domain, {})
            agent_name = domain_config.get("agent_name", f"{domain} 전문가")
            
            formatted.append(
                f"**{agent_name} ({domain})**\n"
                f"신뢰도: {response.confidence_score:.2f}\n"
                f"답변: {response.answer}\n"
                f"출처: {len(response.source_documents)}개 문서"
            )
        
        return "\n\n".join(formatted)
    
    def integrate_responses(self, 
                          question: str,
                          responses: Dict[str, RAGResponse],
                          integration_method: str = "auto") -> IntegratedResponse:
        """응답 통합 수행"""
        
        start_time = datetime.now()
        
        if not responses:
            return IntegratedResponse(
                final_answer="죄송합니다. 답변을 생성할 수 없습니다.",
                contributing_domains=[],
                individual_responses={},
                integration_method="none",
                overall_confidence=0.0,
                source_summary=[],
                processing_time=0.0
            )
        
        # 통합 방식 결정
        if integration_method == "auto":
            if len(responses) == 1:
                integration_method = "single"
            elif len(responses) <= 2:
                integration_method = "combined"
            else:
                integration_method = "ranked"
        
        try:
            formatted_responses = self._format_responses(responses)
            
            if integration_method == "single":
                # 단일 응답 선택
                if len(responses) == 1:
                    final_answer = list(responses.values())[0].answer
                else:
                    final_answer = self.single_chain.invoke({
                        "question": question,
                        "responses": formatted_responses
                    })
            
            else:
                # 복합 통합
                final_answer = self.integration_chain.invoke({
                    "question": question,
                    "responses": formatted_responses
                })
            
            # 전체 신뢰도 계산
            confidence_scores = [r.confidence_score for r in responses.values()]
            overall_confidence = sum(confidence_scores) / len(confidence_scores)
            
            # 출처 요약
            source_summary = []
            for domain, response in responses.items():
                source_count = len(response.source_documents)
                if source_count > 0:
                    agent_name = DOMAIN_CONFIG.get(domain, {}).get("agent_name", domain)
                    source_summary.append(f"{agent_name}: {source_count}개 문서")
            
            processing_time = (datetime.now() - start_time).total_seconds()
            
            return IntegratedResponse(
                final_answer=final_answer,
                contributing_domains=list(responses.keys()),
                individual_responses=responses,
                integration_method=integration_method,
                overall_confidence=overall_confidence,
                source_summary=source_summary,
                processing_time=processing_time
            )
            
        except Exception as e:
            logger.error(f"응답 통합 실패: {e}")
            # 첫 번째 응답을 기본값으로 사용
            first_response = list(responses.values())[0]
            
            return IntegratedResponse(
                final_answer=first_response.answer,
                contributing_domains=list(responses.keys()),
                individual_responses=responses,
                integration_method="fallback",
                overall_confidence=first_response.confidence_score,
                source_summary=[f"단일 응답: {len(first_response.source_documents)}개 문서"],
                processing_time=(datetime.now() - start_time).total_seconds()
            )

# 응답 통합기 초기화
if llm:
    response_integrator = ResponseIntegrator(llm)
    print("✅ 응답 통합기 초기화 완료")
else:
    response_integrator = None
    print("❌ LLM이 초기화되지 않아 응답 통합기를 생성할 수 없습니다.")

## 5. LangGraph 워크플로우 구성

In [ ]:
# LangGraph 상태 정의
class RAGWorkflowState(TypedDict):
    """RAG 워크플로우 상태"""
    question: str
    question_analysis: Optional[QuestionAnalysis]
    selected_agents: List[str]
    agent_responses: Dict[str, RAGResponse]
    integrated_response: Optional[IntegratedResponse]
    error_message: Optional[str]
    processing_start_time: Optional[datetime]

class CoolStayRAGWorkflow:
    """꿀스테이 RAG 워크플로우 관리자"""
    
    def __init__(self, 
                 question_analyzer: QuestionAnalyzer,
                 agents: Dict[str, MockRAGAgent],
                 response_integrator: ResponseIntegrator):
        
        self.question_analyzer = question_analyzer
        self.agents = agents
        self.response_integrator = response_integrator
        
        # 워크플로우 그래프 구성
        self._build_workflow()
    
    def _build_workflow(self):
        """워크플로우 그래프 구성"""
        
        # StateGraph 생성
        workflow = StateGraph(RAGWorkflowState)
        
        # 노드 추가
        workflow.add_node("analyze_question", self._analyze_question)
        workflow.add_node("select_agents", self._select_agents)
        workflow.add_node("execute_agents", self._execute_agents)
        workflow.add_node("integrate_responses", self._integrate_responses)
        
        # 엣지 추가
        workflow.set_entry_point("analyze_question")
        workflow.add_edge("analyze_question", "select_agents")
        workflow.add_edge("select_agents", "execute_agents")
        workflow.add_edge("execute_agents", "integrate_responses")
        workflow.add_edge("integrate_responses", END)
        
        # 메모리 설정 (선택사항) - checkpointer 없이 컴파일
        # memory = MemorySaver()
        
        # 워크플로우 컴파일 (checkpointer 제거)
        self.workflow = workflow.compile()
    
    def _analyze_question(self, state: RAGWorkflowState) -> RAGWorkflowState:
        """질문 분석 단계"""
        try:
            logger.info("질문 분석 시작")
            
            analysis = self.question_analyzer.analyze_question(state["question"])
            
            state["question_analysis"] = analysis
            state["processing_start_time"] = datetime.now()
            
            logger.info(f"질문 분석 완료: {len(analysis.relevant_domains)}개 도메인 식별")
            
        except Exception as e:
            logger.error(f"질문 분석 실패: {e}")
            state["error_message"] = f"질문 분석 실패: {e}"
        
        return state
    
    def _select_agents(self, state: RAGWorkflowState) -> RAGWorkflowState:
        """에이전트 선택 단계"""
        try:
            logger.info("에이전트 선택 시작")
            
            analysis = state.get("question_analysis")
            if not analysis:
                # 기본 에이전트 선택
                selected_agents = ["hr_policy"]
            else:
                # 우선순위 도메인을 기반으로 선택
                selected_agents = analysis.priority_domains[:3]  # 최대 3개 에이전트
                
                # 웹 검색이 필요한 경우 추가
                if analysis.needs_web_search:
                    selected_agents.append("web_search")
            
            # 사용 가능한 에이전트만 필터링
            available_agents = []
            for agent in selected_agents:
                if agent in self.agents or agent == "web_search":
                    available_agents.append(agent)
            
            state["selected_agents"] = available_agents
            
            logger.info(f"에이전트 선택 완료: {available_agents}")
            
        except Exception as e:
            logger.error(f"에이전트 선택 실패: {e}")
            state["selected_agents"] = ["hr_policy"]  # 기본 에이전트
        
        return state
    
    def _execute_agents(self, state: RAGWorkflowState) -> RAGWorkflowState:
        """에이전트 실행 단계"""
        try:
            logger.info("에이전트 실행 시작")
            
            question = state["question"]
            selected_agents = state.get("selected_agents", [])
            responses = {}
            
            for agent_name in selected_agents:
                if agent_name == "web_search":
                    # 웹 검색 에이전트 처리 (Mock)
                    responses[agent_name] = RAGResponse(
                        answer=f"웹 검색 결과: {question}에 대한 최신 정보를 검색했습니다.",
                        source_documents=[],
                        domain=agent_name,
                        confidence_score=0.7
                    )
                elif agent_name in self.agents:
                    # 일반 RAG 에이전트 실행
                    agent = self.agents[agent_name]
                    response = agent.query(question, enable_corrective=True)
                    responses[agent_name] = response
                    
                logger.info(f"{agent_name} 에이전트 실행 완료")
            
            state["agent_responses"] = responses
            
            logger.info(f"모든 에이전트 실행 완료: {len(responses)}개 응답")
            
        except Exception as e:
            logger.error(f"에이전트 실행 실패: {e}")
            state["error_message"] = f"에이전트 실행 실패: {e}"
        
        return state
    
    def _integrate_responses(self, state: RAGWorkflowState) -> RAGWorkflowState:
        """응답 통합 단계"""
        try:
            logger.info("응답 통합 시작")
            
            question = state["question"]
            agent_responses = state.get("agent_responses", {})
            
            if not agent_responses:
                state["error_message"] = "통합할 응답이 없습니다."
                return state
            
            # 응답 통합 수행
            integrated_response = self.response_integrator.integrate_responses(
                question=question,
                responses=agent_responses,
                integration_method="auto"
            )
            
            state["integrated_response"] = integrated_response
            
            logger.info("응답 통합 완료")
            
        except Exception as e:
            logger.error(f"응답 통합 실패: {e}")
            state["error_message"] = f"응답 통합 실패: {e}"
        
        return state
    
    def process_question(self, question: str) -> IntegratedResponse:
        """질문 처리 (전체 워크플로우 실행)"""
        
        # 초기 상태 설정
        initial_state = {
            "question": question,
            "question_analysis": None,
            "selected_agents": [],
            "agent_responses": {},
            "integrated_response": None,
            "error_message": None,
            "processing_start_time": None
        }
        
        try:
            # 워크플로우 실행 (checkpointer가 없으므로 config 불필요)
            result = self.workflow.invoke(initial_state)
            
            # 결과 반환
            if result.get("integrated_response"):
                return result["integrated_response"]
            else:
                # 오류 발생 시 기본 응답
                error_msg = result.get("error_message", "알 수 없는 오류")
                return IntegratedResponse(
                    final_answer=f"죄송합니다. 질문 처리 중 오류가 발생했습니다: {error_msg}",
                    contributing_domains=[],
                    individual_responses={},
                    integration_method="error",
                    overall_confidence=0.0,
                    source_summary=[],
                    processing_time=0.0
                )
                
        except Exception as e:
            logger.error(f"워크플로우 실행 실패: {e}")
            return IntegratedResponse(
                final_answer=f"죄송합니다. 시스템 오류가 발생했습니다: {e}",
                contributing_domains=[],
                individual_responses={},
                integration_method="system_error",
                overall_confidence=0.0,
                source_summary=[],
                processing_time=0.0
            )

# 워크플로우 생성
if all([question_analyzer, mock_agents, response_integrator]):
    rag_workflow = CoolStayRAGWorkflow(
        question_analyzer=question_analyzer,
        agents=mock_agents,
        response_integrator=response_integrator
    )
    print("✅ RAG 워크플로우 생성 완료")
else:
    rag_workflow = None
    missing = []
    if not question_analyzer: missing.append("질문 분석기")
    if not mock_agents: missing.append("에이전트")
    if not response_integrator: missing.append("응답 통합기")
    print(f"❌ RAG 워크플로우를 생성할 수 없습니다. 누락: {', '.join(missing)}")

## 6. 통합 워크플로우 테스트

In [ ]:
def test_integrated_workflow(workflow: CoolStayRAGWorkflow, test_questions: List[str]):
    """통합 워크플로우 테스트"""
    
    print("🚀 통합 RAG 워크플로우 테스트 시작\n")
    
    test_results = []
    
    for i, question in enumerate(test_questions, 1):
        print(f"📋 테스트 {i}: {question}")
        print("="*60)
        
        try:
            start_time = datetime.now()
            
            # 통합 워크플로우 실행
            result = workflow.process_question(question)
            
            end_time = datetime.now()
            total_time = (end_time - start_time).total_seconds()
            
            # 결과 출력
            print(f"\n💬 최종 답변:")
            print(result.final_answer[:400] + "..." if len(result.final_answer) > 400 else result.final_answer)
            
            print(f"\n📊 처리 정보:")
            print(f"   - 참여 도메인: {', '.join(result.contributing_domains)}")
            print(f"   - 통합 방식: {result.integration_method}")
            print(f"   - 전체 신뢰도: {result.overall_confidence:.2f}")
            print(f"   - 처리 시간: {total_time:.2f}초")
            
            if result.source_summary:
                print(f"   - 출처 요약: {', '.join(result.source_summary)}")
            
            # 개별 응답 요약
            if result.individual_responses:
                print(f"\n🤖 개별 에이전트 응답:")
                for domain, response in result.individual_responses.items():
                    agent_name = DOMAIN_CONFIG.get(domain, {}).get("agent_name", domain)
                    print(f"   - {agent_name}: 신뢰도 {response.confidence_score:.2f}")
            
            # 테스트 결과 저장
            test_result = {
                "question": question,
                "domains_used": len(result.contributing_domains),
                "integration_method": result.integration_method,
                "confidence": result.overall_confidence,
                "processing_time": total_time,
                "success": True
            }
            test_results.append(test_result)
            
            print(f"\n✅ 테스트 완료")
            
        except Exception as e:
            print(f"❌ 테스트 실패: {e}")
            logger.error(f"워크플로우 테스트 실패 ({question}): {e}")
            
            test_results.append({
                "question": question,
                "domains_used": 0,
                "integration_method": "error",
                "confidence": 0.0,
                "processing_time": 0.0,
                "success": False
            })
        
        print("\n" + "="*70 + "\n")
    
    # 전체 테스트 결과 요약
    print(f"📈 전체 테스트 결과 요약")
    print(f"   - 총 테스트: {len(test_results)}개")
    successful_tests = [r for r in test_results if r['success']]
    print(f"   - 성공한 테스트: {len(successful_tests)}개")
    print(f"   - 성공률: {len(successful_tests)/len(test_results)*100:.1f}%")
    
    if successful_tests:
        avg_confidence = sum(r['confidence'] for r in successful_tests) / len(successful_tests)
        avg_processing_time = sum(r['processing_time'] for r in successful_tests) / len(successful_tests)
        avg_domains = sum(r['domains_used'] for r in successful_tests) / len(successful_tests)
        
        print(f"\n📊 평균 성능 지표:")
        print(f"   - 평균 신뢰도: {avg_confidence:.2f}")
        print(f"   - 평균 처리 시간: {avg_processing_time:.2f}초")
        print(f"   - 평균 사용 도메인: {avg_domains:.1f}개")
    
    return test_results

# 통합 워크플로우 테스트 실행
workflow_test_questions = [
    "연차 휴가 신청 방법을 알려주세요",
    "시스템 아키텍처와 배포 방법을 설명해주세요",
    "회사 정책에 대해 전반적으로 설명해주세요"
]

if rag_workflow:
    workflow_test_results = test_integrated_workflow(rag_workflow, workflow_test_questions)
    print("\n✅ 통합 워크플로우 테스트 완료!")
else:
    print("❌ RAG 워크플로우가 생성되지 않아 테스트를 건너뜁니다.")

## 7. 요약 및 다음 단계

### ✅ 완료된 작업
1. **질문 분석 시스템**: AI 기반 도메인 분류 및 에이전트 선택
2. **멀티 에이전트 라우팅**: 최적의 전문가 조합 자동 선택
3. **응답 통합 시스템**: 단일/복합 답변 지능형 통합
4. **LangGraph 워크플로우**: 전체 파이프라인 오케스트레이션
5. **통합 테스트**: 엔드투엔드 시스템 검증

### 🔧 핵심 기능
- **Intelligent Routing**: 질문 내용 기반 최적 에이전트 자동 선택
- **Multi-Agent Orchestration**: 1-3개 도메인 전문가 동적 조합
- **Response Integration**: 단일 선택, 복합 통합, 순위 기반 통합
- **Workflow Management**: LangGraph 기반 상태 관리 파이프라인
- **Error Handling**: 각 단계별 오류 처리 및 복구

### 📊 시스템 특징
- **확장 가능**: 새로운 도메인/에이전트 쉽게 추가
- **유연한 통합**: 질문 복잡도에 따른 적응형 응답 생성
- **상태 추적**: 전체 처리 과정 모니터링 가능
- **신뢰도 기반**: 에이전트별 신뢰도 계산 및 가중 평균

### 🚀 다음 단계
**05_hitl_evaluation.ipynb**: Human-in-the-Loop 평가 시스템
- ReAct 평가 에이전트 구현
- 6차원 품질 평가 시스템
- 인터럽트 기반 인간 검증
- 품질 기반 피드백 루프